In [15]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

from flask_cors import CORS, cross_origin
from flask import Flask, jsonify, request
from flask import Flask, render_template, request
from keras.models import load_model
from keras.optimizers import SGD


import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework import ops
import numpy as np
import pandas as pd
import pickle
import json
import random



In [16]:
data = pickle.load(open( "chappy-bot-data.pkl", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data ['train_y']

with open('intents.json') as json_data:
    intents = json.load(json_data)
# model = load_model('chappy-bot-model.h5')

In [17]:
model = keras.Sequential()
model.add(keras.layers.Dense(128, input_shape=(len (train_x[0]),), activation ='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [18]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words


def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)

    for s in sentence_words: 
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return (np.array(bag))

In [19]:
def predict_class(sentence, model):
    p = bow (sentence, words,show_details=False)
    res = model.predict(np.array([p])) [0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
        return return_list


In [20]:
p = bow("I love you", words)
print(p)
print(classes)

found in bag: i
found in bag: lov
found in bag: you
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
['Hate', 'None', 'age', 'banter', 'body-pain', 'breathing', 'cough', 'diarrhoea', 'fever', 'goodbye', 'greeting', 'headache', 'name', 'nasal', 'noanswer', 'options', 'sore-throat', 'thanks', 'tiredness', 'unknown', 'vomit', 'yes']


In [21]:
global graph
graph = ops.get_default_graph()

with open(f'chappy-bot-model.h5', 'rb') as f:
    model = keras.models.load_model('chappy-bot-model.h5')

In [22]:


def getResponse(ints, intents_json):
    responseArray = []
    tag = ints[0] ['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if (i ['tag'] == tag):
            responseArray.append(tag);
            print(i['tag'])
            print(tag)
            
            result = random.choice(i['responses'])
            responseArray.append(result);           
            print (result)
            break
    
    
    
    return responseArray



def chatbot_response(text):
    ints = predict_class(text, model)
    print(ints)
    res = getResponse (ints, intents)
    return res

def chatbot_prediction(text):

    predict_list = text.split(',')
    predict_test = [predict_list]
    loaded_model = pickle.load(open('knnpickle_file', 'rb'))
    result = loaded_model.predict(predict_test) 
    print(result)

    return result




In [23]:
predict_class('I hate you', model)

[{'intent': 'Hate', 'probability': '1.0'}]

In [24]:
predict_class('Hello', model)

[{'intent': 'greeting', 'probability': '1.0'}]

In [25]:
predict_class('I have a cough', model)

[{'intent': 'cough', 'probability': '1.0'}]

In [26]:
predict_class('Yes, I have a cough', model)

[{'intent': 'cough', 'probability': '1.0'}]

In [27]:
predict_class('now', model)

[{'intent': 'unknown', 'probability': '0.5108906'}]

In [14]:
app = Flask(__name__)
app.static_folder = 'static'

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
# def classify():
    
    userText = request.args.get('msg')
    return jsonify(chatbot_response(userText))
    # return str()
    # return str("Hello")

@app.route("/post",methods = ['POST'])
def post_bot_response():
# def classify():
    
    userText = request.form.get('msg')
    return str(chatbot_prediction(userText)[0])   

if __name__ == "__main__":
    app.run() 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Apr/2021 21:20:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 21:20:04] "GET /static/styles/style.css HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 21:20:17] "GET /get?msg=Hi HTTP/1.1" 200 -
[{'intent': 'greeting', 'probability': '1.0'}]
greeting
greeting
Hi there, how can I help?
127.0.0.1 - - [06/Apr/2021 21:20:30] "GET /get?msg=How%20are%20you%3F HTTP/1.1" 200 -
[{'intent': 'greeting', 'probability': '1.0'}]
greeting
greeting
Hello!
127.0.0.1 - - [06/Apr/2021 21:21:01] "GET /get?msg=I%20love%20you%20 HTTP/1.1" 200 -
[{'intent': 'Hate', 'probability': '0.9998286'}]
Hate
Hate
That's Okay, I still care about you!
127.0.0.1 - - [06/Apr/2021 21:21:19] "GET /get?msg=I%20love%20you HTTP/1.1" 200 -
[{'intent': 'Hate', 'probability': '0.9998286'}]
Hate
Hate
I'm just trying to hel